# Team 24 - Nonlinear Time-Transient Rotational Test Rig

From: https://www.compumag.org/wp/wp-content/uploads/2018/06/problem24.pdf


In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import *

## Geometry

In [ ]:
symmetric = True

mm = 1e-3

r_outer = 209*mm/2
r_inner = r_outer - 21.4*mm
height = 25.4*mm
stator_ring = Cylinder((0,0,-height/2),Z,r_outer,height) - Cylinder((0,0,-height/2),Z,r_inner,height, mantle="cyl_inner")
stator_block = Box((-27.8/2*mm, -r_inner, -height/2), (27.8/2*mm, r_inner, height/2))

# Set "helper" facenames to easily find fillet edges later
stator_block.faces.Min(X).name = "block"
stator_block.faces.Max(X).name = "block"
stator_block -= Cylinder((0,0,-height/2),Z,107.5/2*mm, height)
stator = stator_ring + stator_block
edges = stator.faces["block"].edges * stator.faces["cyl_inner"].edges
stator = stator.MakeFillet(edges, 3*mm)

# Reset helper face names
stator.faces["cyl_inner|block"].name = None
stator.faces.col = (0.8,0.8,0.8)
stator.solids.name = "stator"

r_inner = 50.8*mm/2
r_outer = r_inner + 12.7*mm
rotor_ring = Cylinder((0,0,-height/2),Z,r_outer,height, mantle="cyl_outer") - Cylinder((0,0,-height/2),Z,r_inner,height)
rotor_block = Box((-25.4*mm/2, -100*mm, -height/2), (25.4*mm/2, 100*mm, height/2)) * Cylinder((0,0,-height/2),Z,102.1*mm/2, height, mantle="rotor_end")
rotor_block.faces.Min(X).name = "block"
rotor_block.faces.Max(X).name = "block"
rotor_block -= Cylinder((0,0,-height/2),Z,r_inner, height)
rotor = rotor_ring + rotor_block
edges = rotor.faces["block"].edges * rotor.faces["cyl_outer"].edges
rotor = rotor.MakeFillet(edges, 3*mm)
rotor.faces.col = (0,0.37, 0.42)

search_coil_y = rotor.vertices.Max(Y).p[1]
search_coil = Glue([Segment((-25.4*mm/2, search_coil_y-8.7*mm, -height/2), (25.4*mm/2, search_coil_y-8.7*mm, -height/2)),
                    Segment((25.4*mm/2, search_coil_y-8.7*mm, -height/2), (25.4*mm/2, search_coil_y-8.7*mm, height/2)),
                    Segment((25.4*mm/2, search_coil_y-8.7*mm, height/2), (-25.4*mm/2, search_coil_y-8.7*mm, height/2)),
                    Segment((-25.4*mm/2, search_coil_y-8.7*mm, height/2), (-25.4*mm/2, search_coil_y-8.7*mm, -height/2))])
search_coil.edges.name = "search_coil"
search_coil.edges.col = (1,0,0)
rotor = Glue([rotor, search_coil])
rotor = rotor.Rotate(Axis((0,0,0), Z), -22)
rotor.faces["cyl_outer|block"].name = None
rotor.solids.name = "rotor"


coil = Box((-34/2*mm-24*mm, -17*mm/2, -24*mm-31*mm/2), (34/2*mm+24*mm, 17*mm/2, 31*mm/2 + 24*mm))
coil -= Box((-34/2*mm, -17*mm/2, -31*mm/2), (34/2*mm, 17*mm/2, 31*mm/2))
edges = ListOfShapes([coil.edges.Min(X+Z),coil.edges.Max(X+Z),coil.edges.Min(X-Z),coil.edges.Max(X-Z)])
coil = coil.MakeFillet(edges, 24*mm)
coil.faces.col = (0.72,0.45,0,2)
cut1 = HalfSpace((-34/2*mm, 0,0), X)
cut2 = HalfSpace((34/2*mm, 0,0), X)
cut1.faces.col = (0,0,0,0)
cut2.faces.col = (0,0,0,0)
c1 = coil * cut1
c2 = coil - cut1
c2.solids.name = "coil_mid"
c1.solids.name = "coil_left"
coil = Glue([c1, c2])
c3 = coil - cut2
c3.solids.name = "coil_right"
coil = Glue([coil*cut2, c3])
cut3 = HalfSpace((0, 0,-31*mm/2), Z)
cut4 = HalfSpace((0, 0,31*mm/2), Z)
cut3.faces.col = (0,0,0,0)
cut4.faces.col = (0,0,0,0)
c4 = coil * cut3
for s in c4.solids:
    s.name += "_bottom"
c5 = coil - cut3
c6 = c5 * cut4
for s in c6.solids:
    s.name += "_mid"
c7 = c5 - cut4
for s in c7.solids:
    s.name += "_top"
coil = Glue([c4, c6, c7])
coil.solids.maxh=0.01

coil1 = coil.Move((0,63*mm,0))
for s in coil1.solids:
    s.name += "_1"
coil2 = coil.Move((0,-63*mm,0))
for s in coil2.solids:
    s.name += "_2"

airbox_size = 400*mm
airbox_height = 300*mm
air = Box((-airbox_size/2, -airbox_size/2, -airbox_height/2), (airbox_size/2, airbox_size/2, airbox_height/2))
air.faces.name = "outer"
air.faces.col = (0,0,0.3,0.2)
air.solids.name = "air"
shape = Glue([stator, rotor, coil1, coil2, air])

if symmetric:
    sym = HalfSpace((0,0,0), (0,0,1))
    sym.faces.name = "sym"
    sym.faces.col = (0.5,0.5,0.5,0.5)
    shape = shape * sym
shape.WriteStep("motor.step")
Draw(shape, euler_angles=(-50,0,30))

## Mesh

In [ ]:
geo = OCCGeometry(shape)
pole_end = (0.0139, 0.0519216, -0.0127)
shift = (-6.5e-3, -1.3e-3, 7.7e-3)
hall_point = tuple(p + s for p, s in zip(pole_end, shift))
mp = meshsize.moderate
mp.RestrictH(*hall_point, 0.0001)
mesh = Mesh(geo.GenerateMesh(mp, closeedgefac=0.1, maxh=0.02))
mesh.Curve(5)
print("Number of elements:", mesh.ne)
Draw(mesh, euler_angles=(-50,0,30), clipping={"z":-1, "dist":0.1})

## Current Density

In [ ]:
coilrect_xmin, coilrect_xmax = -34*mm/2, 34*mm/2
coilrect_zmin, coilrect_zmax = -31*mm/2, 31*mm/2

Jtau = mesh.MaterialCF({ "coil_left_bottom.*" : (coilrect_zmin-z, 0, x-coilrect_xmin),
                        "coil_mid_bottom.*" : (1,0,0),
                        "coil_right_bottom.*" : (coilrect_zmin-z, 0, x-coilrect_xmax),
                        "coil_right_mid.*" : (0,0,1),
                        "coil_left_mid.*" : (0,0,-1),
                        "coil_left_top.*" : (coilrect_zmax-z, 0, x-coilrect_xmin),
                        "coil_mid_top.*" : (-1,0,0),
                        "coil_right_top.*" : (coilrect_zmax-z, 0, x-coilrect_xmax)},
                       default=(0,0,0))

J_coil = Jtau * 1/Norm(Jtau)
Draw(J_coil, mesh, clipping={"pnt":(0,63*mm,0), "vec": (0,1,0)}, euler_angles=[-60,0,20], draw_surf=False, vectors={"grid_size":400})

## FESpace

In [ ]:
graddoms = [ 1 if mat in ("stator", "rotor") else 0 for mat in mesh.GetMaterials() ]
order = 3
hcurl = HCurl(mesh, order=order, dirichlet="sym", gradientdomains=graddoms)
number = NumberSpace(mesh, definedon=mesh.Materials("coil.*"))
fes = hcurl * number
(u, I), (v, It) = fes.TnT()

# Material Parameters

In [ ]:
mu0 = 1.257e-6

B = [0, 1.413, 1.594, 1.751, 1.839, 1.896, 1.936, 1.967, 2.008, 2.042, 2.073, 2.101, 2.127, 2.151, 2.197, 2.240, 2.281, 2.321, 2.361, 2.400, 2.472]
h = [0, 00.400, 00.801, 01.601, 02.402, 03.203, 04.003, 04.804, 06.405, 08.007, 09.608, 11.210, 12.811, 14.412, 17.615, 20.818, 24.020, 27.223, 30.426, 33.629, 39.634]
# extend data by high value
H = [1e4*hi for hi in h]
H.append(H[-1] + (H[-1]-H[-2])/(B[-1]-B[-2])*(1000-B[-1]))
B.append(1000)

bh = BSpline(2, [0] + B, H)
import matplotlib.pyplot as plt
import numpy as np
# test evaluation of bspline
bvals = np.linspace(0, 3, 1000)
plt.plot([bh(bi) for bi in bvals], bvals)

B = curl(u)
normB = sqrt(B * B + 1e-20)
nu = mesh.MaterialCF( { "rotor|stator" : bh(normB)/normB }, default=1/mu0)

sigma = mesh.MaterialCF( {"stator|rotor" : 4.54e5 }, default=0)

N = 350 # number of turns
S = 24*17*mm**2  # coil cross-section

In [ ]:
tau = 0.005
uold = GridFunction(hcurl)

a = BilinearForm(fes)
a += nu * (curl(u) * curl(v) + 1e-6 * u * v) * dx
a += sigma/tau * (u-uold) * v * dx
a += -N/S * I * J_coil * v * dx("coil.*", bonus_intorder=30)


voldom = Integrate(1, mesh.Materials("coil.*"))
# actually \int E = \int (u-uold)/tau must be multiplied by 2
# multiplying the voldom is the same and keeps symmetry
if symmetric:
    voldom *= 2 # for symmetry

U = 23.1
R = 3.09 # Ohm

a += tau/voldom * (-R*I-U) * It * dx("coil.*", bonus_intorder=5)
a += - N/S * J_coil * (u-uold) * It * dx("coil.*", bonus_intorder=6)
c = Preconditioner(a, type="bddc", inverse="sparsecholesky")
# c = Preconditioner(a, type="direct", inverse="pardiso")

u = GridFunction(fes)
uprime = GridFunction(hcurl)

uold.vec[:] = 0.0

res = u.vec.CreateVector()
w = u.vec.CreateVector()

times = [0]
By_vals = [0]
I_vals = [0]
rotor_flux = [0]
torque = [0]

In [ ]:


from IPython.display import display, clear_output
measured_t = [0, 0.002, 0.005, 0.010, 0.015, 0.020,
              0.030, 0.040, 0.050, 0.060, 0.070, 0.080,
              0.090, 0.100, 0.120, 0.140, 0.160, 0.180,
              0.200, 0.240, 0.300]
measured_By = [0, 0.07, 0.21, 0.38, 0.53, 0.64, 0.82, 0.94, 1.03,
               1.09, 1.14, 1.17, 1.19, 1.21, 1.23, 1.24, 1.25,
               1.25, 1.25, 1.26, 1.26]
measured_I = [0, 0.91, 1.80, 2.95, 3.82, 4.49, 5.45, 6.05, 6.45,
              6.71, 6.91, 7.04, 7.15, 7.22, 7.31, 7.36, 7.38, 7.40,
              7.40, 7.41, 7.41]
measured_rotor_flux = [0, 0.29, 0.77, 1.44, 1.99, 2.44, 3.11, 3.58, 3.91, 4.14, 4.31, 4.43, 4.52, 4.58, 4.65, 4.69, 4.71, 4.72, 4.72, 4.72, 4.72]
measured_rotor_flux = [1e-4*val for val in measured_rotor_flux]
measured_torque = [0, 0.02, 0.17, 0.39, 0.68, 0.96, 1.48, 1.9, 2.24, 2.48, 2.68, 2.82, 2.93, 3.02, 3.11, 3.17, 3.19, 3.21, 3.22, 3.22, 3.22]


scene = Draw(curl(u.components[0]), mesh, draw_surf=False, clipping={"pnt" : (0,0,-0.1*mm), "vec" : (0,0,-1)}, vectors={"grid_size":300})
scene_J = Draw(sigma*uprime, mesh, draw_surf=False, clipping={"pnt" : (0,0,-0.1*mm),"vec" : (0,0,-1)}, vectors={"grid_size":500})

In [ ]:
from ngsolve.nonlinearsolvers import NewtonSolver
from ngsolve.krylovspace import CGSolver
tend = 0.3
hall_probe = mesh(*hall_point)

plt.ion()
fig, axs = plt.subplots(4, 1, constrained_layout=True)
# relative error
plot_by, = axs[0].plot(times, By_vals, label="computed")
axs[0].plot(measured_t, measured_By, label="measured", linestyle=":")
axs[0].set_title("By at hall probe")
axs[0].legend()

plot_i, = axs[1].plot(times, I_vals, label="computed")
axs[1].plot(measured_t, measured_I, label="measured", linestyle=":")
axs[1].set_title("I")
axs[1].legend()

plot_flux, = axs[2].plot(times, rotor_flux, label="computed")
axs[2].plot(measured_t, measured_rotor_flux, label="measured", linestyle=":")
axs[2].set_title("Rotor flux")
axs[2].legend()

plot_torque, = axs[3].plot(times, torque, label="computed")
axs[3].plot(measured_t, measured_torque, label="measured", linestyle=":")
axs[3].set_title("Rotor torque")
axs[3].legend()

plt.ioff()
plt.show()
with TaskManager():
    t = 0
    a.AssembleLinearization(u.vec)
    solver = CGSolver(a.mat, c, maxiter=1000, tol=1e-4, printrates=True)
    newton = NewtonSolver(a, u, solver=solver)
    while t < tend:
        t = t + tau
        uold.vec.data = u.components[0].vec
        newton.Solve(maxerr=1e-8, printing=True)
        uprime.vec.data = 1/tau * u.components[0].vec - 1/tau * uold.vec
        times.append(t)
        B = curl(u.components[0])
        by = B(hall_probe)[1]
        I = -u.components[1].vec[0]
        flux_rotor = Integrate(u.components[0] * specialcf.tangential(3), mesh, definedon=mesh.BBoundaries("search_coil"))
        B_vol = BoundaryFromVolumeCF(mesh.MaterialCF({"air": B}))
        n = specialcf.normal(mesh.Materials("rotor"))
        rotation = CF((-y,x,0))
        maxwell_stress = 1/mu0 * (OuterProduct(B_vol, B_vol) - 0.5 * (B_vol * B_vol) * Id(3))
        torq = Integrate(maxwell_stress * n * rotation, mesh, definedon=mesh.Materials("rotor").Boundaries() * mesh.Materials("air").Boundaries())
        if symmetric:
            flux_rotor *= 2
            torq *= 2
        torque.append(torq)
        rotor_flux.append(flux_rotor)
        By_vals.append(by)
        I_vals.append(I)
        plot_by.set_ydata(By_vals)
        plot_i.set_ydata(I_vals)
        plot_flux.set_ydata(rotor_flux)
        plot_torque.set_ydata(torque)
        for plot in (plot_by, plot_i, plot_flux, plot_torque):
            plot.set_xdata(times)
        for ax in axs:
            ax.relim()
        plt.draw()
        clear_output(wait=True)
        print("t = ", t)
        print("By = ", by)
        print("I = ", I)
        print("rotor flux = ", flux_rotor)
        print("torque: ", torque[-1])
        display(fig)
        scene.Redraw()
        scene_J.Redraw()
